# Warning please use virtual env if dont have fun errors

# Install required dependencies

In [2]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:06
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:01:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 781.0 kB/s eta 0:00:0031m41.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━

# Importing the dependecies

In [2]:
import cv2 # type: ignore
import numpy as np # type: ignore
import os 
from matplotlib import pyplot as plt # type: ignore
import time
import mediapipe as mp # type: ignore

2024-09-14 10:47:26.998362: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-14 10:47:27.260760: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-14 10:47:27.615761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 10:47:27.889576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 10:47:27.994594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-14 10:47:28.508091: I tensorflow/core/platform/cpu_feature_gu

# Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [30]:
cap = cv2.VideoCapture(0)
# Set media pipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        # Read feed
        ret, frame = cap.read()
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        # draw_landmarks(image, results)
        draw_styled_landmarks(image, results)
        
        # Show to screen
        cv2.imshow('Sign language detection', image)
        
        # Terminate program
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1726291952.508313  135093 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1726291952.510657  136765 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) HD Graphics 520 (SKL GT2)
W0000 00:00:1726291952.726565  136761 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726291952.817889  136759 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726291952.820706  136762 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726291952.822261  136760 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback t

# Extract Keypoints values

In [31]:
len(results.pose_landmarks.landmark)

33

In [32]:

pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test) 

In [43]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)

In [44]:
pose.shape, left_hand.shape, right_hand.shape

((132,), (63,), (63,))

In [45]:
pose

array([ 5.38359344e-01,  3.39714497e-01, -8.95556927e-01,  9.99946833e-01,
        5.63827574e-01,  2.64041573e-01, -8.14892769e-01,  9.99904335e-01,
        5.80285728e-01,  2.62579173e-01, -8.14602971e-01,  9.99908805e-01,
        5.96130490e-01,  2.62194306e-01, -8.13925147e-01,  9.99895096e-01,
        5.06263077e-01,  2.74577737e-01, -8.34044814e-01,  9.99907374e-01,
        4.83849823e-01,  2.80554622e-01, -8.32928181e-01,  9.99907553e-01,
        4.63007987e-01,  2.87505627e-01, -8.33535016e-01,  9.99904752e-01,
        6.18046761e-01,  2.95175850e-01, -3.70785773e-01,  9.99917626e-01,
        4.35310781e-01,  3.17920327e-01, -4.49173778e-01,  9.99949217e-01,
        5.72513223e-01,  4.25408006e-01, -7.39091516e-01,  9.99959111e-01,
        5.05310297e-01,  4.34960783e-01, -7.64142096e-01,  9.99967337e-01,
        7.79115081e-01,  6.84736073e-01, -1.97872207e-01,  9.99804854e-01,
        2.78946221e-01,  6.76627040e-01, -2.58307129e-01,  9.99178529e-01,
        1.05548644e+00,  

In [46]:
face

array([ 0.54238755,  0.41056949, -0.03195506, ...,  0.6242131 ,
        0.25411949,  0.02413429])

In [39]:
left_hand

array([ 8.85786533e-01,  6.02120161e-01,  4.36515194e-07,  8.02830935e-01,
        5.84703743e-01, -2.62328256e-02,  7.30945885e-01,  5.29658794e-01,
       -3.95872183e-02,  6.79946184e-01,  4.85104322e-01, -5.16660623e-02,
        6.35198176e-01,  4.56421852e-01, -6.41095415e-02,  7.74031699e-01,
        3.66275847e-01, -1.73795354e-02,  7.39424884e-01,  2.74845839e-01,
       -3.41628343e-02,  7.16493249e-01,  2.20903933e-01, -5.03450930e-02,
        6.96576715e-01,  1.74304947e-01, -6.37653768e-02,  8.21716547e-01,
        3.42696398e-01, -2.40348335e-02,  7.99062967e-01,  2.27308407e-01,
       -3.67263705e-02,  7.84149051e-01,  1.55497000e-01, -5.18586710e-02,
        7.71376908e-01,  9.69935656e-02, -6.45048022e-02,  8.71346951e-01,
        3.45809668e-01, -3.55069600e-02,  8.61825764e-01,  2.32723042e-01,
       -5.43356091e-02,  8.54586959e-01,  1.62459061e-01, -7.08596259e-02,
        8.45148981e-01,  9.88062322e-02, -8.36738274e-02,  9.21588778e-01,
        3.68705213e-01, -

In [40]:
right_hand

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

If the landmark is not found

In [41]:
np.zeros(21*3).shape

(63,)

In [50]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    
    return np.concatenate([pose, face, left_hand, right_hand])

In [61]:
result_test = extract_keypoints(results)

In [62]:
result_test

array([ 0.53835934,  0.3397145 , -0.89555693, ...,  0.        ,
        0.        ,  0.        ])

In [63]:
468 * 3 + 33 * 4 + 2 *(21 * 3)

1662

In [64]:
np.save('0', result_test)

In [65]:
np.load('0.npy')

array([ 0.53835934,  0.3397145 , -0.89555693, ...,  0.        ,
        0.        ,  0.        ])

# Setup Folders for Collection Data set

In [55]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Dataset')

# Actions that we try to detect
actions = np.array(['hello', 'thank you', 'good', 'afternoon'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequences_length = 30

In [66]:
# Creating a folder for each action like : hello, thank you, good, afternoon

for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collecting Keypoints values for training and testing

In [67]:
cap = cv2.VideoCapture(0)
# Set media pipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequences_length):
                # Read feed
                ret, frame = cap.read()
                
                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                
                # Draw landmarks
                # draw_landmarks(image, results)
                draw_styled_landmarks(image, results)
                
                # Apply collection wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 1, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    # Show to screen
                    cv2.imshow('Sign language detection', image)
                    cv2.waitKey(2000) #! 2 seconds wait time (changeable for the higher sign language moments)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('Sign language detection', image)
                    
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
        
                # Terminate program
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1726294994.461728  135093 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1726294994.471414  138913 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) HD Graphics 520 (SKL GT2)
W0000 00:00:1726294994.726144  138908 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726294994.788216  138908 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726294994.791665  138907 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726294994.792324  138909 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback t

# Preprocess Data and Create Lables and Features

In [68]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [69]:
label_map = {label:num for num, label in enumerate(actions)}

In [70]:
label_map

{'hello': 0, 'thank you': 1, 'good': 2, 'afternoon': 3}

### Labeling the image

In [71]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequences_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [74]:
np.array(sequences).shape

(120, 30, 1662)

In [75]:
np.array(labels).shape

(120,)

In [76]:
X = np.array(sequences)

In [77]:
X.shape

(120, 30, 1662)

In [78]:
y = to_categorical(labels).astype(int)

In [79]:
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [121]:
y_test.shape

(6, 4)

# Build and Train LSTM Neural Network

In [129]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [130]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir) 

In [131]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) #! 30 changes according to the sequence length
model.add(LSTM(128, return_sequences=True, activation='relu')) 
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu')) 
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

/home/vaibhav/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [132]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [133]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - categorical_accuracy: 0.2863 - loss: 4.8726
Epoch 2/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - categorical_accuracy: 0.2601 - loss: 21.7096
Epoch 3/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - categorical_accuracy: 0.2211 - loss: 4.1269
Epoch 4/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - categorical_accuracy: 0.3899 - loss: 4.6734
Epoch 5/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - categorical_accuracy: 0.1845 - loss: 4.5518
Epoch 6/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - categorical_accuracy: 0.2651 - loss: 5.3116
Epoch 7/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - categorical_accuracy: 0.3003 - loss: 4.2348
Epoch 8/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - categorical_accuracy: 0.1272 - loss: 5.1501
Epoch 9/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - categorical_accuracy: 0.2575 - loss: 2.5037
Epoch 10/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - categorical_accuracy: 0.2441 - loss: 2.9298
Epoch 11/1000
4/4 ━━━━

KeyboardInterrupt: 

In [134]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 30, 64)         │       442,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,790,126 (6.83 MB)

 Trainable params: 596,708 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,193,418 (4.55 MB)

# Make Predictions

In [135]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 547ms/step


In [141]:
actions[np.argmax(res[4])]

'afternoon'

In [142]:
actions[np.argmax(y_test[4])]

'afternoon'

# Save Weights

In [138]:
model.save('sign_language_model.h5')

In [139]:
model.load_weights('sign_language_model.h5')

# Evaluation using Confusion Matrix and Accuracy

In [143]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [144]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


In [145]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [146]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [1, 2]],

       [[4, 1],
        [0, 1]],

       [[4, 0],
        [0, 2]]])

In [147]:
accuracy_score(ytrue, yhat)

0.8333333333333334

# Test in Real Time

In [153]:
# Detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.8     #! Threshold for prediction probability (How much model is sure about the prediction)

cap = cv2.VideoCapture(0)
# Set media pipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        # Read feed
        ret, frame = cap.read()
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        # draw_landmarks(image, results)
        draw_styled_landmarks(image, results)
        
        # Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
        # Screen logic
            if np.unique(predictions[-10:])[0] == np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)        
        
        # Show to screen
        cv2.imshow('Sign language detection', image)
        
        # Terminate program
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1726304227.033638  135093 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1726304227.035220  203168 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) HD Graphics 520 (SKL GT2)
W0000 00:00:1726304227.171678  203164 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726304227.223392  203165 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726304227.225169  203164 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726304227.225874  203163 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback t

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
good
1/1 ━━━━━━━